In [2]:
import sys
sys.path.append('../')
import os
import pickle
import tqdm
import numpy as np
import pandas as pd
import seaborn as sns   
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
import modules.comparison as comparison
import matplotlib.patches as patches
import matplotlib.colors as col
from colorspacious import cspace_convert  
import json


similarity_threshold = 0.998
eps = 0.12

## 1. Collect the maps from the folder "maps"

In [5]:
# create the list of maps and exclude some if needed
#code to create the list of maps and to print the number of georeferenced maps out of the total

MAIN_FOLDER = './maps/' # FOLDER PATH
map_folders = os.listdir(MAIN_FOLDER) # READ THE FOLDER PATH AND GET THE LIST OF THE FOLDERS INSIDE
map_folders = [x for x in map_folders if x != '.DS_Store'] # REMOVE THE .DS_STORE FILE
                 
map_list = []
for folder in map_folders:
    if os.path.exists(MAIN_FOLDER+folder+'/'):
        map_files = [x for x in os.listdir(MAIN_FOLDER+folder+'/') if x != '.DS_Store'] # REMOVE THE .DS_STORE FILE
        objects_to_collect = {} # dictionary to store the map subfiles to collects
        year = folder.split('_')[0]
        auth = folder.split('_')[1]
        name_parts = folder.split('_')
        
        if len(name_parts) == 3:
            try:
                int(name_parts[2])
                number = name_parts[2]
                name = auth + '_' + year + '_' + number
            except:
                name = auth + '_' + name_parts[2] + '_' + year
        else:
            name = auth + '_' + year
  
        for file in map_files:
            if file.endswith(name + '.png') or file.endswith(name + '.jpg') or file.endswith(name + '.tif') or file.endswith(name + '.pdf') or file.endswith(name + '.jpeg') or file.endswith(name + '.tiff'):
                image_path = MAIN_FOLDER+folder+'/'+file
                objects_to_collect['image_path'] = image_path

            if file.endswith('.points'):
                file_path = MAIN_FOLDER+folder+'/'+file
                first_row = pd.read_csv(file_path, nrows=1, delimiter=',')
                points = pd.read_csv(file_path, skiprows=1, delimiter = ',') 
                crs_info = str(first_row.columns)
                epsg_code = comparison.extract_epsg(crs_info)  # Extract EPSG code using the new function

                #append the image to the image list while associating it with the folder name
                objects_to_collect['points'] = points
                objects_to_collect['folder'] = folder
                objects_to_collect['folder_path']=MAIN_FOLDER+folder+'/'
                objects_to_collect['epsg'] = epsg_code
            if file.endswith('metadata.json'):
                #read the json metadata file
                
                with open(MAIN_FOLDER+folder+'/'+file) as json_file:
                    try:
                        metadata = json.load(json_file)
                        objects_to_collect['metadata'] = metadata
                    except:
                        continue
        if len(objects_to_collect) == 6:
            
            map_list.append(objects_to_collect)  
            #print(objects_to_collect['folder'])


## 2. Create the map objects:

In [6]:
#CODE TO CREATE THE MAP objects
grid_size = 20
maps = [comparison.create_map_object(map_info, grid_size) for map_info in map_list]

## 3. Create graphs for all maps: